<a href="https://colab.research.google.com/github/kaggle-challenge-group/machine_learning_model/blob/main/kaggle_machine_lrnmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# # load the dataset from the google drive
# from google.colab import drive
# drive.mount('/content/drive')
# import multiprocessing
# img_count = multiprocessing.cpu_count()

In [35]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

In [36]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
print(f"Full train dataset shape is {train.shape}")
test = pd.read_csv('/content/drive/MyDrive/test.csv')
print(f"Full train dataset shape is {test.shape}")

Full train dataset shape is (8693, 14)
Full train dataset shape is (4277, 13)


In [37]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [38]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [39]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [40]:
def return_no_null_values(df):
  return df.isnull().sum()

In [41]:
return_no_null_values(train)


PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [42]:
return_no_null_values(test)

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [43]:
def null_values_percentage(df):
  return return_no_null_values(df)/ len(df) * 100

In [44]:
null_values_percentage(test)


PassengerId     0.000000
HomePlanet      2.034136
CryoSleep       2.174421
Cabin           2.338087
Destination     2.151040
Age             2.127660
VIP             2.174421
RoomService     1.917232
FoodCourt       2.478373
ShoppingMall    2.291326
Spa             2.361468
VRDeck          1.870470
Name            2.197802
dtype: float64

In [45]:
df = train.copy()
dft = test.copy()


In [46]:
def convert_bool(col):
    if col is True:
        return 1
    elif col is False:
        return 0
    else:
        return col  # Keeps NaN as is


In [47]:
def preprocessing(df,bool_cols,columns_to_impute):
  df.drop(['PassengerId', 'Name'], axis=1, inplace=True)
  df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)
  df.drop(['Cabin'], axis=1, inplace=True)
  # Apply conversion
  for col in bool_cols:
      df[col] = df[col].apply(convert_bool)
  df['Side'] = df['Side'].map({'P': 1, 'S': 0, np.nan: np.nan})
  df['Num'] = pd.to_numeric(df['Num'], errors='coerce')
  deck_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 8}
  df['Deck'] = df['Deck'].map(deck_mapping)
  imputer = IterativeImputer(max_iter=10, random_state=0)
  df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])
  df['Deck'] = df['Deck'].round().astype(int)
  # For HomePlanet
  train_data_homeplanet = df[df['HomePlanet'].notna()]
  predict_data_homeplanet = df[df['HomePlanet'].isna()]

  # For Destination
  train_data_destination = df[df['Destination'].notna()]
  predict_data_destination = df[df['Destination'].isna()]

  # Extract features and target for HomePlanet
  X_homeplanet = train_data_homeplanet.drop(['HomePlanet', 'Destination'], axis=1)
  y_homeplanet = train_data_homeplanet['HomePlanet']

  # Extract features for prediction set
  X_predict_homeplanet = predict_data_homeplanet.drop(['HomePlanet', 'Destination'], axis=1)

  # Label encode the target
  le_homeplanet = LabelEncoder()
  y_homeplanet_encoded = le_homeplanet.fit_transform(y_homeplanet)

  # Initialize the model
  model_homeplanet = RandomForestClassifier(random_state=0)

  # Train the model
  model_homeplanet.fit(X_homeplanet, y_homeplanet_encoded)

  # Predict missing HomePlanet values
  predicted_homeplanet_encoded = model_homeplanet.predict(X_predict_homeplanet)

  # Decode predictions back to original labels
  predicted_homeplanet = le_homeplanet.inverse_transform(predicted_homeplanet_encoded)

  # Fill in the predicted values
  df.loc[df['HomePlanet'].isna(), 'HomePlanet'] = predicted_homeplanet

  # Extract features and target for Destination
  X_destination = train_data_destination.drop(['HomePlanet', 'Destination'], axis=1)
  y_destination = train_data_destination['Destination']

  # Extract features for the prediction set where Destination is missing
  X_predict_destination = predict_data_destination.drop(['HomePlanet', 'Destination'], axis=1)

  # Label encode the target for Destination
  le_destination = LabelEncoder()
  y_destination_encoded = le_destination.fit_transform(y_destination)

  # Initialize the model for Destination
  model_destination = RandomForestClassifier(random_state=0)

  # Train the model
  model_destination.fit(X_destination, y_destination_encoded)

  # Predict missing Destination values
  predicted_destination_encoded = model_destination.predict(X_predict_destination)

  # Decode predictions back to original labels for Destination
  predicted_destination = le_destination.inverse_transform(predicted_destination_encoded)

  # Fill in the predicted values in the original DataFrame
  df.loc[df['Destination'].isna(), 'Destination'] = predicted_destination

  encoder = OneHotEncoder(sparse=False)

  # Columns to be one-hot encoded
  columns_to_encode = ['HomePlanet', 'Destination']

  # Apply OneHotEncoder
  encoded_data = encoder.fit_transform(df[columns_to_encode])

  encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

  # Concatenate the original DataFrame (minus the encoded columns) with the new encoded DataFrame
  df_final = pd.concat([df.drop(columns_to_encode, axis=1), encoded_df], axis=1)
  return df_final


In [48]:
columns_to_impute = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Num', 'Deck', 'Side','CryoSleep','VIP']
bool_cols = ['VIP', 'CryoSleep', 'Transported']

In [49]:
train_data = preprocessing(df,bool_cols,columns_to_impute)
train_data.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1,6,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [50]:
test_data = preprocessing(dft,bool_cols[:-1],columns_to_impute)
test_data.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,7,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0,6,4.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,38.0,0.0,0.0,6652.0,0.0,181.0,585.0,3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,20.0,0.0,10.0,0.0,635.0,0.0,0.0,6,5.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [51]:

return_no_null_values(train_data)

CryoSleep                    0
Age                          0
VIP                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
Transported                  0
Deck                         0
Num                          0
Side                         0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
dtype: int64

In [52]:
return_no_null_values(test_data)

CryoSleep                    0
Age                          0
VIP                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
Deck                         0
Num                          0
Side                         0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
dtype: int64

In [53]:
X = train_data.drop('Transported', axis=1)
y = train_data['Transported']

In [54]:
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

RandomForestClassifier(random_state=42)

In [55]:
predicted_transported = model.predict(test_data)

In [56]:
# Prepare the submission DataFrame
submission_df = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Transported': predicted_transported.astype(bool)  # Ensure the boolean type if your model outputs integers
})

# Save the submission file
submission_df.to_csv('submission.csv', index=False)

In [57]:
submission_df.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [58]:
sample_submission_df = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [59]:
from sklearn.metrics import accuracy_score

# Merge the sample submission with your predictions to align them
comparison_df = sample_submission_df.merge(submission_df, on='PassengerId', suffixes=('_sample', '_predicted'))

# Calculate accuracy
accuracy = accuracy_score(comparison_df['Transported_sample'], comparison_df['Transported_predicted'])
print(f"Accuracy: {accuracy}")

Accuracy: 0.5295768061725509
